In [1]:
import logging
import sys

import matplotlib.pyplot as plt

root = logging.getLogger()
root.setLevel(logging.INFO)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)

In [2]:
import sys

sys.path.append('../scripts')
sys.path.append('../')
from scripts import centroids_graph_builder
from time import sleep
from multiprocessing import Pool

import numpy as np
from tqdm.notebook import trange, tqdm
from scripts import graph_osm_loader, utils

import networkx as nx
import osmnx as ox
from heapq import heappop, heappush
from itertools import count
from scripts import clustering

In [3]:
from scripts import pfa
from scripts.path_findings import ch_pfa, dijkstra_pfa, ch_builder

In [4]:
def get_rand_graph(N, p):
    G = nx.fast_gnp_random_graph(N, p, directed=False)
    if not nx.is_connected(G):
        tmp = []
        for n in nx.connected_components(G):
            for q in n:
                tmp.append(q)
                break
        for i in range(len(tmp) - 1):
            G.add_edge(tmp[i], tmp[i + 1])
    for e in G.edges:
        G.add_edge(e[0], e[1], length=np.random.random_sample() + 0.001)
    for u in G.nodes:
        if u in G[u]:
            G.remove_edge(u, u)
    return G

In [5]:
NUM_ITERATION = 10  # чтобы уменьшить ошибку при вычислении времени выполнения, при каждом замере время меряется для NUM_ITERATION повторений
WORKER = 5  # количество потоков

In [6]:
GRAPH_ID = 'R2555133'  # R13470549 R2555133 R3766483
# примеры id есть в graph_osm_loader.py
# g = get_rand_graph(1000, 0.1)  # загрузка графа
N = 10000
p = 3 / N
# g = get_rand_graph(N,p)
# g = get_graph()
g = graph_osm_loader.get_graph(GRAPH_ID)
print(len(g.nodes), len(g.edges))

17940 27061


In [7]:
g.remove_edges_from(nx.selfloop_edges(g))

In [8]:
pfa = dijkstra_pfa.Dijkstra(g = g)

In [9]:
# points = [(u,v) for u in g.nodes() for v in g.nodes() if u!=v]
points = [utils.get_node_for_initial_graph_v2(g) for _ in trange(100, desc='generate points')]

generate points:   0%|          | 0/100 [00:00<?, ?it/s]

In [10]:
pfa.find_path(*points[0])

(np.float64(16436.019605159538),
 [6100270055,
  376304913,
  312013064,
  312013065,
  260122293,
  4890491633,
  319388692,
  251542746,
  314943660,
  255743902,
  255743900,
  251542444,
  267379882,
  324532006,
  301117073,
  490992191,
  1086168550,
  324532022,
  179536100,
  179572305,
  179533092,
  339066654,
  271890579,
  10929975439,
  10929975434,
  7519693097,
  919567892,
  2536195935,
  5064247362,
  528559357,
  85689991,
  4409390073,
  4409390071,
  85690065,
  965792146,
  314969137,
  306558382,
  470248651,
  598963408,
  12289960409,
  265694621,
  2379495376,
  251068505,
  251068504,
  257747436,
  251068496,
  1460826461,
  274854237,
  253033871,
  458231744,
  458231746,
  274854332,
  458231749,
  6646242555,
  252851864,
  253027649,
  253033642,
  458231752,
  313295946,
  313295947,
  252845726,
  340157261,
  315056701,
  282654209,
  315056702,
  253067715,
  458100699,
  458109135,
  253948013,
  306081981,
  95949502,
  95949504,
  339937106,
  959

In [11]:
nx.single_source_dijkstra(g, *points[0], weight='length')

(np.float64(16436.019605159538),
 [6100270055,
  376304913,
  312013064,
  312013065,
  260122293,
  4890491633,
  319388692,
  251542746,
  314943660,
  255743902,
  255743900,
  251542444,
  267379882,
  324532006,
  301117073,
  490992191,
  1086168550,
  324532022,
  179536100,
  179572305,
  179533092,
  339066654,
  271890579,
  10929975439,
  10929975434,
  7519693097,
  919567892,
  2536195935,
  5064247362,
  528559357,
  85689991,
  4409390073,
  4409390071,
  85690065,
  965792146,
  314969137,
  306558382,
  470248651,
  598963408,
  12289960409,
  265694621,
  2379495376,
  251068505,
  251068504,
  257747436,
  251068496,
  1460826461,
  274854237,
  253033871,
  458231744,
  458231746,
  274854332,
  458231749,
  6646242555,
  252851864,
  253027649,
  253033642,
  458231752,
  313295946,
  313295947,
  252845726,
  340157261,
  315056701,
  282654209,
  315056702,
  253067715,
  458100699,
  458109135,
  253948013,
  306081981,
  95949502,
  95949504,
  339937106,
  959

In [ ]:
ch = ch_builder.GreedyBuilder().build_ch_pfa(g)

build ch graph:   0%|          | 0/17940 [00:00<?, ?it/s]

In [ ]:
ch.find_path(*points[0])

In [ ]:
cms = clustering.resolve_k_means_communities(g, resolution=10, max_iteration=100, cluster_name='cluster',
                                             print_log=True)
print(len(cms))

In [ ]:
cls2hubs = {}
cls2n = centroids_graph_builder.get_cls2n(g)
for i, c in enumerate(cms):
    for j in cls2n[i]:
        if (i, j) in cls2hubs:
            continue
        a = set()
        b = set()
        for u in c:
            for v in g[u]:
                if v in cms[j]:
                    a.add(u)
                    b.add(v)
        cls2hubs[i, j] = a if len(a) < len(b) else b
        cls2hubs[j, i] = a if len(a) < len(b) else b

In [ ]:
cls2hub = {}  # = centroids_graph_builder.get_cls2hubs(g)
for i, j in cls2hubs:
    if i not in cls2hub:
        cls2hub[i] = set()
    cls2hub[i].update(cls2hubs[i, j])
del cls2hubs

In [ ]:
np.mean([len(c) for c in cls2hub.values()])

In [ ]:
hubs = set([u for v in cls2hub.values() for u in v])

In [ ]:
hub2id = {h: i for i, h in enumerate(hubs)}
id2hub = {i: h for i, h in enumerate(hubs)}

In [ ]:
len(set(hubs))

In [ ]:
def dijkstra_pfa_to_set(graph: nx.Graph,
                        start: int,
                        ends: set[int]
                        ) -> \
        tuple[float, list[int]]:
    adjacency = graph._adj
    c = count()
    push = heappush
    pop = heappop
    dist = {}
    fringe = []

    dist[start] = (0.0, None)
    push(fringe, (0.0, next(c), start))
    visited = set()
    
    while fringe:
        (d, _, v) = pop(fringe)
        if v in ends:
            visited.add(v)
        if len(visited) == len(ends):
            break
        
        for u, e in adjacency[v].items():
            vu_dist = d + e['length']
            if u not in dist or dist[u][0] > vu_dist:
                dist[u] = (vu_dist, v)
                push(fringe, (vu_dist, next(c), u))
    return dist


In [ ]:
def cals(data_partitions):
    part, hubs, l = data_partitions
    dst = np.zeros((l, l))

    for u in part:
        dst_u = dijkstra_pfa_to_set(g, u, hubs)
        for v in hubs:
            dst[hub2id[u], hub2id[v]] = dst_u[v][0]
    return dst


data = [(list(hubs)[i::WORKER], hubs, len(hubs)) for i in range(WORKER)]
with Pool(WORKER) as p:
    dst = sum(tqdm(p.imap_unordered(cals, data), total=len(data)))

In [ ]:
def cals(data_partitions):
    part, l = data_partitions
    res = {}
    for u in part:
        dst_u = dijkstra_pfa_to_set(g, u, cls2hub[g.nodes()[u]['cluster']])
        for v in cls2hub[g.nodes()[u]['cluster']]:
            node = v
            path = [node]
            while dst_u[node][1]is not None:
                node = dst_u[node][1]
                path.append(node)
            res[u, v] = dst_u[v][0]#, set(g.nodes()[p]['cluster'] for p in path)
    return res

data = [(list(g.nodes)[i::WORKER], hubs) for i in range(WORKER)]
with Pool(WORKER) as p:
    d_nodes = {k: v for d in tqdm(p.imap_unordered(cals, data), total=len(data)) for k, v in d.items()}

In [ ]:
from pympler.asizeof import asizeof
asizeof(d_nodes)/1024/1024, asizeof(dst)/1024/1024

In [ ]:
all_paths = dict(nx.all_pairs_dijkstra_path_length(g, weight='length'))

In [ ]:
nodes = g.nodes()
    

In [ ]:
def dijkstra_pfa_cls(g, u, v):
    # return all_paths[u][v] ,[]
    c1, c2 = nodes[u]['cluster'], nodes[v]['cluster']
    if c1 == c2:
        return 0,[]
    return min(d_nodes[u, h1] + d_nodes[v, h2] + dst[hub2id[h1], hub2id[h2]] for h1 in cls2hub[c1] for h2 in cls2hub[c2]), []

In [ ]:
def dijkstra_pfa(graph: nx.Graph,
                 start: int,
                 end: int) -> \
        tuple[float, list[int]]:
    if start == end:
        return 0, [start]
    adjacency = graph._adj
    nodes = graph.nodes()
    c = count()
    push = heappush
    pop = heappop
    dist = {}
    pred = {}
    fringe = []
    push(fringe, (0,0.0, next(c), 0, start, None))
    while fringe:
        (_,d, _, n, v, p) = pop(fringe)
        if v in dist:
            continue
        dist[v] = (d, n)
        pred[v] = p
        if v == end:
            break
        for u, e in adjacency[v].items():
            
            l = dijkstra_pfa_cls(g, u,end)[0]
            vu_dist = d + e['length']
            
            if u not in dist:
                push(fringe, (vu_dist + l,vu_dist, next(c), n + 1, u, v))
    d, n = dist[end]
    n += 1
    path = [None] * n
    i = n - 1
    e = end
    while i >= 0:
        path[i] = e
        i -= 1
        e = pred[e]
    return d, path


In [ ]:
# points = [(u,v) for u in g.nodes() for v in g.nodes() if u!=v]
points = [utils.get_node_for_initial_graph_v2(g) for _ in trange(100, desc='generate points')]

In [ ]:
@utils.profile(iterations=NUM_ITERATION)
def usual_path(g, p1, p2):
    return nx.single_source_dijkstra(g, p1, p2, weight='length')


@utils.profile(iterations=NUM_ITERATION)
def h_path(g, p1, p2):
    return dijkstra_pfa(g, p1, p2)


def do_calc(data_partitions):
    point_partition, worker_number = data_partitions

    stat = {
        'l': [],
        'h_l': [],
        'p': [],
        'h_p': [],
        'time_l': [],
        'time_h': [],
        'delta': []
    }

    # чисто чтобы tqdm нормально прогрузился 
    sleep(worker_number / 10)
    print('start', worker_number)

    for p1, p2 in tqdm(
            point_partition,
            desc=f'find paths {worker_number}',
            position=worker_number
    ):
        # класический дейкстра
        time_l, (l, p) = usual_path(g, p1, p2)
        # иерархический
        time_h, (h_l, h_p) = h_path(g, p1, p2)
        delta = (h_l - l) / l * 100

        stat['l'].append(l)  # длина обычного пути
        stat['h_l'].append(h_l)  # длина иерархического пути
        stat['p'].append(p)  # обычный путь
        stat['h_p'].append(h_p)  # иерархический путь
        stat['delta'].append(delta)  # разница в длине
        stat['time_l'].append(time_l)  # обычное время 
        stat['time_h'].append(time_h)  # иерархическое
    return stat


data = [([p for p in points[i::WORKER]], i) for i in range(WORKER)]
# do_calc(data[0])
with Pool(WORKER) as p:
    stat = {k: v for r in p.imap_unordered(do_calc, data) for k, v in r.items()}

print(f"err_mean: {np.mean(stat['delta']):.2f} %")
print(f"err_min: {np.min(stat['delta']):.2f} %")
print(f"err_max: {np.max(stat['delta']):.2f} %", )
print(f"acceleration: {np.mean(np.array(stat['time_l']) / np.array(stat['time_h'])):.2f} times")

In [ ]:
acceleration = np.array(stat['time_l']) / np.array(stat['time_h'])
plt.boxplot(acceleration)

In [ ]:
idx = np.argmax(stat['delta'])

In [ ]:
p1,p2 = stat['l'][idx], stat['h_l'][idx]

In [ ]:
p1,p2

In [ ]:
p = stat['p'][idx]
hp = stat['h_p'][idx]
len(p), len(hp)

In [ ]:
u,v = p[0],p[-1]

In [ ]:
dijkstra_pfa(g,u,v)

In [ ]:
u,v

In [ ]:
bi_dijkstra_pfa(g, u,v)